<a href="https://colab.research.google.com/github/Benchamin242/Scraping/blob/main/Scraping2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import requests

In [65]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats" #providing initial link

In [66]:
data = requests.get(standings_url)

In [67]:
from bs4 import BeautifulSoup

In [68]:
soup = BeautifulSoup(data.text) #initialise soup

In [69]:
standings_table = soup.select('table.stats_table')[0] #gave soup details for what to select

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l] #filter the links to link to each team

In [ ]:
links

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links] #give it absolute path

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0] #grabs data from given table and turns HTML into dataframe

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links] #use list comprehension

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l] #specify to only find shooting stats

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}") #gets html of specific link

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0] #turns shooting data into dataframe

In [ ]:
shooting.head()


In [ ]:
shooting.columns = shooting.columns.droplevel() #gets rid of unnecessary level

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date") #merges matches and shooting dataframe

In [ ]:
team_data.head()

In [ ]:
years = list(range(2024, 2022, -1)) #scrapes previous seasons

In [ ]:
years

In [ ]:
all_matches = [] #when loop is done, will contain match logs for one team at end of season

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text) #parse html
    standings_table = soup.select('table.stats_table')[0] #find and intitialise standings_table

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com/{l}" for l in links] #filter our links and add absolute paths

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"


    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # cleans up team name

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0] #parsing scores and fixtures table

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l] #pulling out allcompsshooting to get stats
        data = requests.get(f"https://fbref.com{links[0]}")

        shooting = pd.read_html(data.text, match="Shooting")[0] #read in stats using pandas
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"] #filters to only prem games

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(100)



In [70]:
match_df.columns = [c.lower() for c in match_df.columns]

NameError: name 'match_df' is not defined

In [71]:
match_df

NameError: name 'match_df' is not defined

In [72]:
match_df.to_csv("matches.csv") #put scraped data into csv file

NameError: name 'match_df' is not defined